In [1]:
# has race condition, won't add to 2 million
import threading
import torch

total = torch.tensor(0, dtype=torch.int32)

def count_up(count):
    global total
    for i in range(count):
        total += 1

t1 = threading.Thread(target=count_up, args=[1000000])
t2 = threading.Thread(target=count_up, args=[1000000])
t1.start()
t2.start()
t1.join()
t2.join()

total

tensor(1999578, dtype=torch.int32)

In [2]:
import dis
dis.dis("")


  1           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE


In [3]:
dis.dis("total += 1")

  1           0 LOAD_NAME                0 (total)
              2 LOAD_CONST               0 (1)
              4 INPLACE_ADD
              6 STORE_NAME               0 (total)
              8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


In [4]:
import time
total = torch.tensor(0, dtype=torch.int32)

def count_up(count):
    global total
    for i in range(count):
        total += 1

t1 = threading.Thread(target=count_up, args=[1000000])
t2 = threading.Thread(target=count_up, args=[1000000])
start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print("seconds", end-start)
total

seconds 15.86142897605896


tensor(1999607, dtype=torch.int32)

In [5]:
# add locking
import time

total = torch.tensor(0, dtype=torch.int32)
lock = threading.Lock() # protects total

def count_up(count):
    global total
    for i in range(count):
        lock.acquire()
        total += 1
        lock.release()

t1 = threading.Thread(target=count_up, args=[1000000])
t2 = threading.Thread(target=count_up, args=[1000000])
start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print("seconds", end-start)
total

seconds 32.690959215164185


tensor(2000000, dtype=torch.int32)

In [6]:
# add locking (coarse grained)
import time

total = torch.tensor(0, dtype=torch.int32)
lock = threading.Lock() # protects total

def count_up(count):
    global total
    lock.acquire()
    for i in range(count):
        total += 1
    lock.release()

t1 = threading.Thread(target=count_up, args=[1000000])
t2 = threading.Thread(target=count_up, args=[1000000])
start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print("seconds", end-start)
total

seconds 24.14177966117859


tensor(2000000, dtype=torch.int32)

In [7]:
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    lock.acquire()
    success = False
    if bank_accounts[src] >= amount:
        bank_accounts[src] -= amount
        bank_accounts[dst] += amount
        success = True
    print("transferred" if success else "denied")
    lock.release()

In [8]:
print(bank_accounts)
transfer("x", "y", 3)
print(bank_accounts)

{'x': 25, 'y': 100, 'z': 200}
transferred
{'x': 22, 'y': 103, 'z': 200}


In [9]:
# fix the exception issue where the lock is never released
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    with lock: #   calls lock.acquire() for me
        success = False
        if bank_accounts[src] >= amount:
            bank_accounts[src] -= amount
            bank_accounts[dst] += amount
            success = True
        print("transferred" if success else "denied")
    # calls lock.release() for me

In [10]:
transfer("x", "y", 1)

transferred


In [11]:
transfer("x", "y", 1)

transferred
